In [5]:
import cv2
import os
import mediapipe as mp
import argparse

path = './../../Data/'

In [6]:
#read image 
image_path = os.path.join(path,'images','face.jpg')
image_path_video = os.path.join(path,'videos','testVideo.mp4')



In [7]:
def process_img(img, face_detection):
    H,W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out = face_detection.process(img_rgb)
    
    
    # print(out.detections)
    if out.detections is not None:
        for detection in out.detections:
            location_data = detection.location_data
            bbox = location_data.relative_bounding_box 

            x1, y1, w, h = bbox.xmin, bbox.ymin, bbox.width, bbox.height

            x1 = int(x1 * W)
            y1 = int(y1 * H)
            w = int(w * W)
            h = int(h * H)

            # img = cv2.rectangle(img, (x1, y1), (x1+w, y1+h), (0,255,255), 10)

            # blurring
            if y1 is not None:
                
                img[y1:y1+h, x1:x1+w, :] = cv2.blur(img[y1:y1+h, x1:x1+w, :], (30,30))
    return img

args = argparse.ArgumentParser()

args.add_argument('--mode', default='webcam')
args.add_argument('--filepath', default=image_path)
args.add_argument('--vidfilepath', default=None)

args = args.parse_args()

In [8]:
# Detecting face
mp_face_detection = mp.solutions.face_detection

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:

    if args.mode in ['image']:
        img = cv2.imread(image_path)
      

        img = process_img(img, face_detection)

        # cv2.imshow('image', img)
        # cv2.waitKey(0)  


        cv2.imwrite(os.path.join(path, 'images', 'face_blur_output.png'), img) 
    
    elif args.mode in ['video']:
        cap = cv2.VideoCapture(args.vidfilepath)
        
        ret, frame = cap.read()
        # cv2.imshow('m', frame)
        # cv2.waitKey(0)
        output_video = cv2.VideoWriter(os.path.join(path, 'videos', 'outputvid.mp4'), cv2.VideoWriter_fourcc(*'MP4V'), 25, (frame.shape[1], frame.shape[0]))
        while ret:
            frame = process_img(frame, face_detection)
            output_video.write(frame)
            ret, frame = cap.read()

            # cv2.imshow('frame',frame)
            # if cv2.waitKey(40) & 0xFF == ord('q'):
            #     break

        cap.release()
        output_video.release()
    
    elif args.mode in ['webcam']:
        cap = cv2.VideoCapture(0)
        
        ret, frame = cap.read()
        while ret:
            frame = process_img(frame, face_detection)
           
            cv2.imshow('frame',frame)
            ret, frame = cap.read()

            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

        cap.release()
        

